In [2]:
# importando bibliotecas
from pyspark.sql import SparkSession
import pyspark as ps
spark = (SparkSession
        .builder
        .appName('atv-sala')
        .config('spark.some.config.option', 'some-value')
        .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/05 10:15:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/05 10:15:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# configurando sessao
conf = ps.SparkConf().setMaster("yarn-client").setAppName("spark-mer")
conf.set('spark.executor.heartbeatInterval', '3600s')

In [4]:
spark_realtor = (spark.read.format('csv')
                .option("header", "true")
                .load("realtor-data.csv"))
spark_realtor

DataFrame[status: string, bed: string, bath: string, acre_lot: string, city: string, state: string, zip_code: string, house_size: string, prev_sold_date: string, price: string]

In [5]:
# 1) conte os registros
print(f"Total de registros: {spark_realtor.count()}")

Total de registros: 100000


In [6]:
# verificando tabela
spark_realtor.show(2)

+--------+---+----+--------+--------+-----------+--------+----------+--------------+--------+
|  status|bed|bath|acre_lot|    city|      state|zip_code|house_size|prev_sold_date|   price|
+--------+---+----+--------+--------+-----------+--------+----------+--------------+--------+
|for_sale|3.0| 2.0|    0.12|Adjuntas|Puerto Rico|   601.0|     920.0|          NULL|105000.0|
|for_sale|4.0| 2.0|    0.08|Adjuntas|Puerto Rico|   601.0|    1527.0|          NULL| 80000.0|
+--------+---+----+--------+--------+-----------+--------+----------+--------------+--------+
only showing top 2 rows



In [8]:
# importando funções
import pyspark.sql.functions as f   
from pyspark.sql.functions import col, lit, when

In [9]:
# 2) total de registros no estado de puerto rico com preço maior que 100.000
realtor_pr_100k = spark_realtor.filter((col('state')== 'Puerto Rico') & (col('price') > 100000))
realtor_pr_100k.show(5)

+--------+---+----+--------+-------------+-----------+--------+----------+--------------+--------+
|  status|bed|bath|acre_lot|         city|      state|zip_code|house_size|prev_sold_date|   price|
+--------+---+----+--------+-------------+-----------+--------+----------+--------------+--------+
|for_sale|3.0| 2.0|    0.12|     Adjuntas|Puerto Rico|   601.0|     920.0|          NULL|105000.0|
|for_sale|4.0| 2.0|     0.1|        Ponce|Puerto Rico|   731.0|    1800.0|          NULL|145000.0|
|for_sale|4.0| 3.0|    0.46|San Sebastian|Puerto Rico|   612.0|    2520.0|          NULL|179000.0|
|for_sale|5.0| 3.0|    7.46|   Las Marias|Puerto Rico|   670.0|    5403.0|          NULL|300000.0|
|for_sale|3.0| 2.0|    0.08|   Juana Diaz|Puerto Rico|   795.0|    1045.0|          NULL|150000.0|
+--------+---+----+--------+-------------+-----------+--------+----------+--------------+--------+
only showing top 5 rows



In [10]:
# fazendo o count 
print(f"Número de registros no Estado do Porto Rico com valor maior que 100 mil dolares: {realtor_pr_100k.count()}")

Número de registros no Estado do Porto Rico com valor maior que 100 mil dolares: 17127


In [11]:
realtor_tamanho = spark_realtor.withColumn('tamanho',
                                        when(col('house_size') < 1000, 'pequena')
                                        .when((col('house_size') > 1000) & (col('house_size') < 5000), 'media')
                                        .when(col('house_size') > 5000, 'grande')
                                        .otherwise('sem info'))
realtor_tamanho.show(5)

+--------+---+----+--------+----------+-----------+--------+----------+--------------+--------+--------+
|  status|bed|bath|acre_lot|      city|      state|zip_code|house_size|prev_sold_date|   price| tamanho|
+--------+---+----+--------+----------+-----------+--------+----------+--------------+--------+--------+
|for_sale|3.0| 2.0|    0.12|  Adjuntas|Puerto Rico|   601.0|     920.0|          NULL|105000.0| pequena|
|for_sale|4.0| 2.0|    0.08|  Adjuntas|Puerto Rico|   601.0|    1527.0|          NULL| 80000.0|   media|
|for_sale|2.0| 1.0|    0.15|Juana Diaz|Puerto Rico|   795.0|     748.0|          NULL| 67000.0| pequena|
|for_sale|4.0| 2.0|     0.1|     Ponce|Puerto Rico|   731.0|    1800.0|          NULL|145000.0|   media|
|for_sale|6.0| 2.0|    0.05|  Mayaguez|Puerto Rico|   680.0|      NULL|          NULL| 65000.0|sem info|
+--------+---+----+--------+----------+-----------+--------+----------+--------------+--------+--------+
only showing top 5 rows

